### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 2. Orbits: Finite Coxeter Groups in Action

[![Vitruvian Man](images/vitruvian.jpg)](https://en.wikipedia.org/wiki/Vitruvian_Man)

## Introduction

We have seen how a single algorithm, the **orbit algorithm** (i.e., BFS in the context of group actions), can be used to
* find the orbit $x^G$ of a point $x$ under the action of a group $G$,

and, with appropriate minor modifications and extensions by
* `words`: to express group elements as words in the generators,
* `reps`: to compute a transversal of the point stabilizer,
* `stab`: to assemble Schreier generators of the point stabilizer,
* `edges`: to record the edges of the action graph,
* `images`: to describe the action homomorphism $G \to \mathrm{Aut}(X)$.

We now use the graph traversal techniques from the last day to
* construct a **finite Coxeter group** as a permutation group,
* compute **basic properties** of elements and parabolic subgroups,
* determine the **normalizer** complement of a parabolic subgroup,
* examine the **conjugation action** with respect to element length.

First, reload the algorithms ...

In [ ]:
Read("orbits.g");
LoadPackage("jupyterviz");
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");;

## Finite Coxeter Groups

<div class="alert alert-danger">

**Definition.**

* Let $S$ be a finite set, and let $M = (m_{st})$ be a symmetric matrix with $m_{ss} = 1$
and $m_{st} = m_{ts} \in \{2,3,4,\dots,\infty\}$ if $s \neq t$.
* A **Coxeter group** with  **Coxeter Matrix** $M$ is the group $W$ given by the presentation
    $$
    W = \langle S \mid (st)^{m_{st}} = 1;\, s,t \in S \rangle
    $$
</div>

* The presentation of a Coxeter group can be described by a graph with vertex set $S$ and edges labelled $m_{st}$  between $s$ and $t$ if $m_{st} > 2$, where for $m_{st} = 3$ the label is usually omitted.
* A Coxeter group is called **simply laced** if $m_{st} \leq 3$ for all $s, t \in S$.
* Coxeter graphs (for simply laced irreducible finite Coxeter groups) can be made by the following GAP program.

In [ ]:
coxeterGraph := function(series, rank)
    local edges;
    edges := List([2..rank], j -> [j-1, j]);
    if series >= "D" then  edges[1][2] := 3;  fi;
    if series >= "E" then  edges[2][2] := 4;  fi;
    return edges;
end;

* For example:

In [ ]:
graph := coxeterGraph("A", 7);

In [ ]:
PlotGraph(graph, opts);

* Furthermore, a finite Coxeter group is a (real) **reflection group**, i.e., a group generated by reflections, as follows.
* From the Coxeter matrix $M$, or the corresponding graph, we set up a **Cartan matrix** $C$:
* $C = (c_{st})$ with $c_{ss} = 2$ and $c_{st} c_{ts} = 4 \cos^2 \frac{\pi}{m_{st}}$, $c_{st} = 0 \iff c_{ts} = 0$, $s \neq t$.

In [ ]:
cartanMat := function(series, rank)
    local cartan,  ij,  i,  j;
    cartan := 2*IdentityMat(rank);;
    for ij in coxeterGraph(series, rank) do
        i := ij[1];  j := ij[2];
        cartan[i][j] := -1;
        cartan[j][i] := -1;
    od;
    if series = "B" then  cartan[1][2] := -2;  fi;
    if series = "C" then  cartan[2][1] := -2;  fi;
    if series = "F" then  cartan[3][4] := -2;  fi;  # sic!
    return cartan;
end;

In [ ]:
C := cartanMat("A", 3);;
PrintArray(C);

* From the Cartan matrix $C$ we can compute matrices for the **simple reflections** $s \in S$: the space $V = \mathbb{R}^n$ has a basis of **simple roots** $\{\alpha_s : s \in S\}$ and
$$
\alpha_t.\tilde{s} = \alpha_t - c_{st} \alpha_s
$$
defines a linear action $\tilde{s} \in \mathrm{GL}(V)$ of $s$ as **reflection** in the hyperplane orthogonal to $\alpha_s$.

In [ ]:
S := [1..Length(C)];;  mats := [];;  one := C^0;;
for s in S do
    mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
od;

In [ ]:
for mat in mats do
    PrintArray(mat);  Print("\n");
od;

In [ ]:
mats[1]^2;

* Thus, $W = \langle \tilde{s} : s \in S \rangle \leq \mathrm{GL}(V)$ acts as a **reflection group** on $V$.
* Note that $\{\alpha_s : s \in S\}$ is not the standard basis of $V$.
* But there are real numbers $d_s$, $s \in S$ such that
$$
(\alpha_s, \alpha_t) = \tfrac12 d_s^2 c_{st}
$$
defines a $W$-invariant bilinear form on $V$ (with $d_s = \sqrt{(\alpha_s, \alpha_s)}$).

###  Root System

* The **root system** $\Phi$ of $W$ (or of $C$) is $\Phi = \{\alpha_s.w : s \in S,\,w \in W\}$, a union of $W$-orbits.
* So we can grow this root system as **orbits** of the simple roots $\alpha_s$ (which, as basis vectors of $\mathbb{R}^n$,  are simply the rows of the identity matrix $C^0$).

In [ ]:
phi := orbits(mats, C^0, OnRight);

* Note how $\Phi = \Phi^{+} \cup \Phi^{-}$, where $\Phi^{+} = \{ \alpha \in \Phi : \alpha \geq 0\}$.
* Or, how roots come in pairs of negatives $\{r, -r\}$.
* Thus it suffices to only enumerate the positive representatives of each pair ...

In [ ]:
absRoot := root -> SignInt(Sum(root)) * root;;
onRoots := function(x, a)
    return absRoot(OnRight(x, a));
end;;
phi := orbits(mats, C^0, onRoots);

* ... and find $\Phi$ as concatentation of $\Phi^{+}$ and $- \Phi^{+}$.

In [ ]:
phi := Concatenation(phi, -phi);

* In fact, for future reference, we can and will compute the edges of the action graph, and words in the generators along with $\Phi$. (What are suitable words for the initial roots? Let's take `[i]` for $\alpha_i$.  What are the initial edges?  There are none.)

In [ ]:
orbits_with_words_and_edges := function(aaa, xxx, under)
    local   list,  edges,  words,  i,  k,  l,  z;
    words := List([1..Length(xxx)], i -> [i]);
    list := ShallowCopy(xxx);  edges := [];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(words, onWords(words[i], k));
                l := Length(list);
            fi;
            Add(edges, [i, l]);
        od;
    od;
    return rec(list := list, edges := edges, words := words);
end;

In [ ]:
roots := orbits_with_words_and_edges(mats, C^0, onRoots);;
roots.list;

In [ ]:
edges := Filtered(roots.edges, x-> x[1] <> x[2]);
PlotGraph(edges, opts);

In [ ]:
roots.words;

### Permutations

* Express simple reflections as permutations of the roots. Formula?  (Compare with Schreier generators $(f_y a)/f_{y.a}$!)
* The GAP function `Sortex` returns the permutation needed to sort a list: `Permuted(list, Sortex(list))` is the sorted list.
* So, if $a$ maps `xxx` to `yyy`, $b$ sorts `yyy` (i.e., maps it to `zzz`) and $c$ sorts `xxx` (i.e., maps it to `zzz`) then $c/b$ maps `xxx` to `yyy`.
* The permutation of `xxx` caused is the map that assigns $i \mapsto j$ if `yyy[i] = xxx[j]`, i.e. $b/c = (c/b)^{-1}$ (Check!).

In [ ]:
permutation := function(a, xxx, under)
    return Sortex(List(xxx, x-> under(x, a))) / Sortex(ShallowCopy(xxx));
end;

In [ ]:
permutation(mats[1], phi, OnRight);

* GAP has a function `Permutation` for this purpose.

In [ ]:
Permutation(mats[1], phi, OnRight);

In [ ]:
perms := List(mats, m-> Permutation(m, phi, OnRight));

###  Coxeter Group

* Let's wrap all of the above into a GAP function that constructs a Coxeter group from its Cartan matrix.
* It will be convenient to register an **attribute** `Data` with GAP's permutation groups, as a place to store some Coxeter group data like the root system `phi` and the matrices `mats`. 

In [ ]:
DeclareAttribute("Data", IsPermGroup);

coxeterGroup := function(C)
    local  one,  mats,  roots,  S,  s,  phi,  data,  G;
    one := C^0;  mats := [];  S := [1..Length(C)];
    for s in S do
        mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
    od;
    roots := orbits_with_words_and_edges(mats, C^0, onRoots);
    data := rec(mats := mats, roots := roots, rank := Length(S));
    data.N := Length(roots.list);
    data.phi := Concatenation(roots.list, -roots.list);
    data.perms := List(mats, m -> Permutation(m, data.phi, OnRight));
    G := GroupWithGenerators(data.perms);
    SetData(G, data);
    return G;
end;

In [ ]:
W:= coxeterGroup(C);

In [ ]:
Data(W);

In [ ]:
sizeOfGroup(W);

* $E_8$, for example ...

In [ ]:
E8:= coxeterGroup(cartanMat("E", 8));;
sizeOfGroup(E8);

* ... and its positive roots

In [ ]:
edges := Filtered(Data(E8).roots.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

### Reflections

* From the word $w = a_0 a_1 \dots a_m$ corresponding to the root $\alpha \in \Phi^+$, we compute the reflection
$$
s_{\alpha} = s_{a_0}^{s_{a_{1}} \dotsm s_{a_{m}}} \in W
$$

In [ ]:
reflections := function(W)
    local reflection_from_word;
    reflection_from_word := function(w)
        return W.(w[1])^Product(Data(W).perms{w{[2..Length(w)]}});
    end;
    return List(Data(W).roots.words, reflection_from_word);
end;

* Example:

In [ ]:
A3 := coxeterGroup(cartanMat("A", 3));;
rr := reflections(A3);

In [ ]:
Length(rr);

##  Basic Properties

* Some Basis properties of elements of $W$ are now immediate.

### Length

* The **length** of  an element $w \in W$ is $\ell(w) = \#\{\alpha \in \Phi^{+} : \alpha.w \in \Phi^{-}\}$

In [ ]:
coxeterLength := function(W, w)
    return Number([1..Data(W).N], i-> i^w > Data(W).N);
end;

In [ ]:
List(Data(E8).perms, w-> coxeterLength(E8, w));

### Products

* Conversion `word -> perm`: as a product of simple reflections

In [ ]:
word := [5,2,3,8,7,5,3,1,6];;
perm := Product(Data(E8).perms{word});
coxeterLength(E8, perm);

In [ ]:
permCoxeterWord := function(W, word)
    if word = [] then return (); fi;
    return Product(Data(W).perms{word});
end;

In [ ]:
permCoxeterWord(E8, word);

### Descents

* $s \in S$ is a **left descent** of $w \in W$ if $\ell(sw) < \ell(w)$, i.e., if $\alpha_s.w \in \Phi^{-}$.

In [ ]:
isLeftDescent := function(W, w, s)
    return s^w > Data(W).N;
end;

In [ ]:
isLeftDescent(E8, perm, 5);
isLeftDescent(E8, perm, 2);

### Reduced Expressions

* Conversion `perm -> word`:  $w$ as a word in $S$ is a sequence of left descents.

In [ ]:
coxeterWord := function(W, w)
    local word, a;
    word := [];
    while w <> () do
        a := First([1..Data(W).rank], s-> isLeftDescent(W, w, s));
        Add(word, a);  w := Data(W).perms[a] * w;
    od;
    return word;
end;

In [ ]:
reduced := coxeterWord(E8, perm);

In [ ]:
permCoxeterWord(E8, reduced) = perm;

* To find a reduced expression of any word in $S$, convert `word -> perm` and then `perm -> word`.

In [ ]:
reducedWord := function(W, word)
    return coxeterWord(W, permCoxeterWord(W, word));
end;

In [ ]:
word;
reducedWord(E8, word);

### Longest Elements

* **Longest elements**.  A finite Coxeter group $W$ has a longest element $w_0$.
* In fact, for each $J \subseteq S$ there is one, $w_J$.
* It can be computed as a product of non-descents. 

In [ ]:
longestElement := function(W, J)
    local  wJ,  s;
    wJ := ();
    while true do
        s := First(J, s-> not isLeftDescent(W, wJ, s));
        if s = fail then  return wJ;  fi;
        wJ := W.(s) * wJ;
    od;
end;

In [ ]:
J:= [4,5,6];
wJ:= longestElement(E8, J);
coxeterWord(E8, wJ);

* $w_J$ acts as a **graph automorphism** on $W_J$.

In [ ]:
w6:= longestElement(E8, [1..6]);
Permutation(w6, Data(E8).perms{[1..6]}, OnPoints);

###  Prefixes

* Prefixes (aka weak Bruhat Order): $u \in W$ is a **prefix** of $w \in W$ if $\ell(u) + \ell(u^{-1} w) = \ell(w)$, i.e., if $w = uv$ with $\ell(w) = \ell(u) = \ell(v)$.
The set of all prefixes of $w$ is the **orbit (!)** of $w$ under the action $(w, s) \mapsto ws$ if $\ell(ws) < \ell(w)$, and $w$ else. (Who is acting? On what?)

In [ ]:
prefixes := function(W, w)
    local onRightDescents;
    onRightDescents := function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;
    return orbit([1..Data(W).rank], w, onRightDescents);
end;

* Here, the action function `onRightDescents` is local to the function `prefixes`, since it needs access to the ambient group $W$.

In [ ]:
w := longestElement(E8, [4,5]);;
pre := prefixes(E8, w);;
List(pre, x -> coxeterWord(E8, x));

* The prefixes of $w \in W$ form a graph.

In [ ]:
prefixes_with_edges := function(W, w)
    local onRightDescents;
    onRightDescents := function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;
    return orbit_with_edges([1..Data(W).rank], w, onRightDescents);
end;

In [ ]:
pre := prefixes_with_edges(E8, w);;
PlotGraph(Filtered(pre.edges, x -> x[1] <> x[2]), opts);

### Parabolic Subgroups and Distinguished Coset Representatives

* For a subset $J \subseteq S$, the subgroup $W_J = \langle J \rangle$ is called a **standard parabolic subgroup** of $W$.
* Each coset $W_J w$, $w \in W$, contains a **unique element of minimal length**.
* The subgroup $W_J$ thus has a **distinguished transversal** $X_J = \{ x \in W : \ell(ux) = \ell(u) + \ell(x) \text{ for all } u \in W_J \}$.
* In fact, $X_J$ is the set of prefixes of the **longest coset representative** $d_J = w_J^{-1} w_S = w_J w_S$.

In [ ]:
longestCosetElement := function(W, J, L)
    return longestElement(W, J) * longestElement(W, L);
end;

* For example, the longest coset representative of $E_6 \subset E_7$ (in $E_8$).

In [ ]:
dJL := longestCosetElement(E8, [1..6], [1..7]);;
word := coxeterWord(E8, dJL);

* Naturally, $X_J$ is a graph.

In [ ]:
XJ := prefixes_with_edges(E8, dJL);;
PlotGraph(Filtered(XJ.edges, x -> x[1] <> x[2]), opts);

* And here is a function that computes the distinguished transversal of a parabolic subgroup $W_J$ of $W$:

In [ ]:
parabolicTransversal := function(W, J)
    return prefixes(W, longestCosetElement(J, [1..Data(W).rank]));
end;

##  Shapes: Classes of Parabolic Subgroups

* $W$ acts on its subgroups by conjugations, in particular on its parabolic subgroups.
* The set of standard parabolic subgroups $W_J$, $J \subseteq S$ is thus partitioned into **shapes**: classes of conjugate parabolics.

###  Longest Words Action

* Obviously, if $L = J \cup \{s\}$ for some $s \in S \setminus J$ then conjugation by $d_J^L$ maps $W_J$ to
$W_K$ for some $K \subseteq L$.  In fact, $K = J^{d_J^L}$.
* Define an action of (the free monoid) $S^*$ on the power set $2^S$ as
$$
(J, s) \mapsto J.s := J^{d_J^L}, \quad L = J \cup \{s\}.
$$
* There is an **Orbit-Stabilizer Theorem** for this action.

<div class="alert alert-danger">

**Theorem** (Howlett, ...) Let $(W, S)$ be a finite Coxeter group, and let $S^*$ act on $2^S$ as above. Then:
* The shape $[J] = \{K \subseteq S : W_J \sim W_K\}$ of $J \subseteq S$ is the $S^*$-orbit of $J$ under the above action.
* $N_W(W_J) = W_J \rtimes N_J$, where $N_J = \{x \in X_J : J^x = J\}$ is the stablizer in $W$ of $J$ under this action.
    
</div>

* Let's see this in action, shapes first.
* Actually, the **tack-on action** $(J, s) \mapsto J \cup \{s\}$ and the **take-away action** $(K, s) \mapsto K \setminus \{s\}$ first.

In [ ]:
tackOn := function(x, s)
    return Union(x, [s]);
end;

In [ ]:
K := tackOn([1,3,5], 4);

In [ ]:
takeAway := function(x, s)
    return Difference(x, [s]);
end;

In [ ]:
takeAway(K, 4);

* Using the longest words action on parabolics, we enumerate the shape of $J \subseteq$ as an $S^*$-orbit.
* Here, the action function `onParabolics` is local to the function `shape`, since it needs access to the ambient group $W$.

In [ ]:
shape := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit([1..Data(W).rank], J, onParabolics);
end;

In [ ]:
shape(E8, [1,3]);

### Shapes

 * Denote by $\Lambda = \{[J] : J \subseteq S\}$ the set of all shapes of $W$.
 * Recall (from the exercise in part 1) how the power set $2^S$ is the orbit of $S^*$ under the `takeAway` action.
 * Now, $\Lambda$ is the $S^*$-orbit of the shape $\{S\}$ under that same action.

In [ ]:
shapes := function(W)
    local onShapes, S;
    onShapes := function(x, s)
        return Set(shape(W, takeAway(x[1], s)));
    end;
    S := [1..Data(W).rank];
    return orbit(S, shape(W, S), onShapes);
end;

* $E_8$, for example:

In [ ]:
sss := shapes(E8);;
List(sss, x -> x[1]);
Length(sss);
Sum(sss, Length);

###  Shape Graph

* Naturally, a shape has a graph.

In [ ]:
shape_with_edges := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit_with_edges([1..Data(W).rank], J, onParabolics);
end;

In [ ]:
sh := shape_with_edges(E8, [1,3]);;
edges := Filtered(sh.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

* In order to compute a **transversal** for the shape, we need to modify the algorithm.
* We need to adjust for the fact that a generator $s \in S \subseteq \mathbb{N}$ merely represents a conjugating element $d_K^L \in W$, which depends on the context.
* We might then as well also take into account, that for the current parabolic $K$ only $s \notin K$ needs to be considered as generator.

In [ ]:
shape_with_transversal := function(W, J)
    local   S,  list,  reps,  i,  K,  s,  a,  L;
    S := [1..Data(W).rank];  list := [J];  reps := [()];  i := 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);
            if not L in list then
                Add(list, L);
                Add(reps, reps[i] * a);
            fi;
        od;
    od;
    return rec(list := list, reps := reps);
end;

In [ ]:
List(shape_with_transversal(E8, [1,3]).reps, x -> coxeterWord(E8, x));

##  Normalizers of Parabolic Subgroups

* Recall that $N_W(W_J) = W_J \rtimes N_J$, and let us compute the complement $N_J$ as the **stabilizer** of $J$ in the action of $S^*$ on $2^S$
* For this, we compute a transversal as before, and additionally Schreier generators, one for each non-tree edge in the action graph.
* We distinguish two kinds of Schreier generators: **ears**, if the edge is a loop arising from $K.s = K$, and **eyes**, if the edge closes a proper cycle with $K.s \neq K$.
* Howlett has shown that the **ears** can be regarded as reflection in a natural way, and thus generate a Coxeter group.  And that nontrivial **eyes** are rare.

In [ ]:
parabolicComplement := function(W, J)
    local   S,  list,  reps,  i,  gens,  K,  s,  a,  L,  j;
    S := [1..Data(W).rank];  list := [J];  reps := [()];
    gens := rec(ears := [], eyes := []);
    i := 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);  j := Position(list, L);
            if j = fail then
                Add(list, L);
                Add(reps, reps[i] * a);
            elif j = i then
                AddSet(gens.ears, reps[i] * a * reps[i]^-1);
            else
                AddSet(gens.eyes, reps[i] * a * reps[j]^-1);
            fi;
        od;
    od;
    return gens;
end;

In [ ]:
parabolicComplement(E8, [1,3]);

##  Conjugacy Class by Length

### Minimal Length Representatives

* For a conjugacy class $C$ of $W$ denote by $C_{\mathrm{min}}$ the set of elements of **minimal length** in $C$.
* For $w \in W$ and $s \in S$ denote $w \longrightarrow w^s$ if $\ell(w^s) \leq \ell(w)$ and extend notation to the reflexive and transitive closure.
* A conjugacy class $C$ is called **cuspidal** if $C \cap W_J = \emptyset$ for all $J \subsetneq S$.
* For $w \in W$, denote by $J(w)$ the set of simple reflections $s \in S$ occuring in a reduced expression for $w$.
* The following property of $C_{\mathrm{min}}$ forms the basis of the definition of a square **character table** for the Iwahori-Hecke algebra of $W$ ...

<div class="alert alert-danger">

**Theorem** (Geck-Pfeiffer)**.**  Let $(W, S)$ be a finite Coxeter group.
* $C \longrightarrow C_{\mathrm{min}}$ for each conjugacy class $C$ of $W$: for each $w \in C$ there is a $v \in C_{\mathrm{min}}$ such that $w \longrightarrow v$.
* The set $\{J(w) : w \in C_{\mathrm{min}}\}$ forms a shape of $W$.
* The conjugacy classes of $W$ are parametrized by pairs $([J], D)$ where $[J]$ is shape and $D$ is a cuspidal class of $W_J$.
</div>

* To find the minimal length conjugates of $x \in W$, we modify the orbit algorithm so that it **forgets the current orbit** whenever a shorter conjugate is found.

In [ ]:
minConjugates := function(W, x)
    local   list,  lx,  i,  S,  y,  s,  z,  lz;
    list := [x];  lx := coxeterLength(W, x);
    i := 0;  S := [1..Data(W).rank];
    while i < Length(list) do
        i := i+1;  y := list[i];
        for s in S do
            z := y^W.(s);  lz := coxeterLength(W, z);
            if lz = lx then
                if not z in list then  Add(list, z);  fi;
            elif lz < lx then
                list := [z];  lx := lz;  i := 0;  break;
            fi;
        od;
    od;
    return list;
end;

* test it on a random element

In [ ]:
E7 := coxeterGroup(cartanMat("E", 7));
w := randomGroupElement(E7);;
coxeterWord(E7, w);

In [ ]:
min := minConjugates(E7, w);;
List(min, x -> coxeterLength(E7, x));

### Conjugacy Classes

* We can use this and the fact that the **supports** of $x \in C_{\mathrm{min}}$ form a shape $[J] \in \Lambda$ to compute a **canonical representative** of minimal length for each conjugacy class $C$ of $W$, starting from any element $w \in C$, as follows.

In [ ]:
coxeterMinRep := function(W, w)
    local   v,  K,  sh,  j;
    v := minConjugates(W, w)[1];
    K := Set(coxeterWord(W, v));
    sh := shape_with_transversal(W, K);
    j := PositionMinimum(sh.list);
    return Minimum(minConjugates(W, v^sh.reps[j]));
end;

In [ ]:
coxeterWord(W, coxeterMinRep(E7, w));

*  Acting on the canonical representatives with a generating set that is closed under conjugation, we enumerate **all the conjugacy classes** of $W$.

In [ ]:
coxeterConjugacyClasses := function(W)
    local   onMinReps;
    onMinReps := function(x, a)
        return coxeterMinRep(W, x * a);
    end;
    return orbit(reflections(W), (), onMinReps);
end;

In [ ]:
cc := coxeterConjugacyClasses(E7);;
List(cc, x -> coxeterLength(E7, x));

* Finding the conjugacy classes of $E_8$ takes about 30 minutes ...

## Exercises, etc.

* Check that the function `permutation`, when it is used to map matrices to their effect on the root system, is a homomorphism.

* ($*$) Find an efficient way to enumerate **double cosets** $X_{JK} = X_J \cap X_K^{-1}$, $J, K \subseteq S$.

* Prove that the shapes set $\Lambda$ is an orbit under `takeAway`.

* ($*$) Rewrite `shape_with_edges` as `shape_with_images`, collecting the labelled, directed edge information as image lists.  What are the resulting permutations?

* Prove that the complement $N_J$ is a stabiliser under `onParabolics`.

* Find an example of a parabolic subgroup $W_J$ whose normalizer complement $N_J$ contains a set of eyes that generates a subgroup of size more than $2$.  Is that group elementary abelian?

* **Cyclic Shift Classes.** We say that $w, v \in W$ are **conjugate by cyclic shift** and write $w \leftrightarrow v$ if both $w \longrightarrow v$ and $v \longrightarrow w$.  Show that this relation is the equivalence relation generated by: $ws \leftrightarrow sw$ if $\ell(ws) = \ell(sw)$, for $w \in W$, $s \in S$.  Write a version of the orbit algorithm that computes the cyclic shift class of a given $w \in W$, i.e., the set $\{v \in W : w \leftrightarrow v\}$, together with the edges $vs \leftrightarrow sv$.

In [ ]:
Read("coxeter.g");
Read("shifts.g");
E6 := coxeterGroup(cartanMat("E", 6));

In [ ]:
w := randomGroupElement(E6);;
cyc := cyclic_shifts_with_edges(E6, w);;
edges := Filtered(Set(cyc.edges), x -> x[1] <> x[2]);;
PlotGraph(edges);

* ($*$) Provide a version of the orbit algorithm that, for a given element $w \in W$ computes the partition of its conjugacy class in $W$ into cyclic shift classes, together with with graph induced by the relation $\leftrightarrow$ on the quotient set.

* ($*$) Define an equivalence relation $\approx$ on $W$ as appropriate closure of: $uv \approx vu$ if $\ell(uv) = \ell(u) + \ell(v) = \ell(v) + \ell(u) = \ell(vu)$, $u,v \in W$.  Show that a $\approx$-class is a union of cyclic shift classes.  Find an example of a $\approx$-class that is not a (single) cyclic shift class.

* **Involutions.**  An element $w \in W$ is an **involution** if $w^2 = 1$.  Show that if $w \in W$ is an involution then its cyclic shift class is just $\{w\}$, i.e., that for $s \in S$, either $\ell(sws) <> \ell(w)$ or $sws = w$.

* Let $I(W) = \{w \in W : w^2 = 1\}$ be the set of all involutions in $W$. Show that the monoid $S^*$ acts on $I(W)$ via
$$
w.s = \begin{cases}
ws & \text{if } \ell(sws) = \ell(w),\\
sws & \text{else.}
\end{cases}
$$
Show that $I(W)$ is the orbit of the indentity element $1$ of $W$ under this action.  Use a suitable version of the orbit algorithm to compute $I(W)$ as orbit of $1$.

In [ ]:
Read("coxeter.g");
Read("involution.g");
A3 := coxeterGroup(cartanMat("A", 3));;
invo := involutions(A3);
List(invo, x -> coxeterWord(A3, x));

* Formulate an action (of $S^*$) on the conjugacy classes of involutions of $W$ that can be used to determine all conjugacy classes of involutions of $W$ as orbit of the conjugacy class of the trivial element.  Verify in examples that the conjugacy classes of involutions correspond to the shapes of those parabolic subgroups $W_K$ whose longest element $w_K$ is central in $W_K$.

In [ ]:
Read("coxeter.g");
Read("involution.g");
W := coxeterGroup(cartanMat("A", 3));;
involutionClasses(W);

In [ ]:
Filtered(shapes(W), sh -> longestElement(W, sh[1]) in Centre(Subgroup(W, GeneratorsOfGroup(W){sh[1]})));